<a href="https://colab.research.google.com/github/wsyuei/wsyuei-stock-markets-analytics-zoomcamp-2024/blob/master/Homework%20Submission/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

In [2]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

#Question 1

In [3]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [44]:
ipos_filings = ipo_dfs[0]
ipos_filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           325 non-null    object        
 1   Symbol                325 non-null    object        
 2   Company Name          325 non-null    object        
 3   Price Range           325 non-null    object        
 4   Shares Offered        252 non-null    float64       
 5   IPO Date              325 non-null    datetime64[ns]
 6   Avg_price             258 non-null    float64       
 7   Shares_offered_value  249 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 20.4+ KB


In [5]:
ipos_filings.head(5)

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-


In [41]:
import re

def calculate_avg_price(input_string):
    y_match = re.search(r'(\d+\.\d+)', input_string)
    m_match = re.search(r'- \$(\d+\.\d+)', input_string)

    y_number = float(y_match.group(1)) if y_match else None
    m_number = float(m_match.group(1)) if m_match else None

    if(y_number != None and m_number != None):
      return (y_number + m_number)/2
    else:
      return y_number

# Examples
examples = ["$4.50 - $6.50", "$4.25", "-"]
for example in examples:
    res = calculate_avg_price(example)
    print(f"{example} ==> {res}")

$4.50 - $6.50 ==> 5.5
$4.25 ==> 4.25
- ==> None


In [45]:
from datetime import datetime

# Convert the 'Filing Date' to datetime()
ipos_filings['Filing Date'] = pd.to_datetime(ipos_filings['Filing Date'], format='%b %d, %Y')
# Convert 'Shares Offered' to float64
ipos_filings['Shares Offered'] = pd.to_numeric(ipos_filings['Shares Offered'], errors='coerce')
# Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given).
ipos_filings['Avg_price'] = ipos_filings['Price Range'].apply(lambda x:calculate_avg_price(x))
# Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price"
ipos_filings['Shares_offered_value'] = ipos_filings['Shares Offered'] * ipos_filings['Avg_price']

In [46]:
ipos_filings.head(5)

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,IPO Date,Avg_price,Shares_offered_value
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,2024-05-03,NaN,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,2024-04-29,3.0,11250000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,2024-04-29,10.0,60000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,2024-04-26,10.0,50000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,2024-04-26,NaN,NaN


In [84]:
ipos_filings["Weekday"] = ipos_filings['Filing Date'].dt.dayofweek
ipos_filings["Year"] = ipos_filings['Filing Date'].dt.year

In [85]:
ipos_filings.head(5)

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,IPO Date,Avg_price,Shares_offered_value,Weekday,Year
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,2024-05-03,NaN,NaN,4,2024
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,2024-04-29,3.0,11250000.0,0,2024
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,2024-04-29,10.0,60000000.0,0,2024
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,2024-04-26,10.0,50000000.0,4,2024
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,2024-04-26,NaN,NaN,4,2024


In [90]:
# Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023,
# which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 25 of it is not null.
ipos_filings_df =  ipos_filings[(ipos_filings['Year'] == 2023) & (ipos_filings['Weekday'] == 4)]

In [92]:
ipos_filings_df.shape

(32, 10)

In [99]:
print("${}M".format(round(ipos_filings_df['Shares_offered_value'].sum() / 1000000)))

$286M


#Question 2

In [112]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url1 = "https://stockanalysis.com/ipos/2023/"
response1 = requests.get(url1, headers=headers)

ipo_dfs1 = pd.read_html(response1.text)
ipos_2023 = ipo_dfs1[0]

url2 = "https://stockanalysis.com/ipos/2024/"
response2 = requests.get(url2, headers=headers)

ipo_dfs2 = pd.read_html(response2.text)
ipos_2024 = ipo_dfs2[0]

stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

nvo_df = yf.download(tickers = "NVO",
                     period = "max",
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [113]:
stacked_ipos_df.head(5)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 8, 2024",NNE,Nano Nuclear Energy Inc.,$4.00,-,-
1,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$28.48,19.38%
2,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.10,-21.25%
3,"Apr 25, 2024",RBRK,"Rubrik, Inc.",$32.00,$33.47,5.53%
4,"Apr 25, 2024",LOAR,Loar Holdings Inc.,$28.00,$49.76,77.96%


In [114]:
# Please remove 'RYZB', as it is no longer available on Yahoo Finance.
stacked_ipos_df.loc[stacked_ipos_df['Symbol'] != "RYZB"]

# Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance)
stacked_ipos_df.loc[stacked_ipos_df['Symbol'] == 'IBAC', 'Symbol'] = 'IBACU'

In [116]:
# Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB')
nvo_df_filtered_from_2024 = nvo_df[nvo_df.index < '2024-03-01']